# Problem Statement 

**Data Analysis case study**

I am working on a logistic problem where they have faced cancellation of orders in their deliveries. 
Their main goal is to be the 1st prefered delivery partner in Africa.

Main objective is partnership, bringing together both drivers and businesses for the 24hr delivery services.

**Challenge 1**
1. Analyse order cancellations
2. visualize most important aspects using your prefered method.
3. document steps 
4. Insights on how to reduce number of cancelled orders.

**Challenge 2**

-There's been deliberate deletion of data
1. Use the data 2, missed data to predict cancellation reasons for all orders with missing data.
2. Document steps and method used.

**Challenge 3**

create a presentation for the relevant work done.


# Import packages

In [ ]:
##for faster data processing
import numba as nb


#for data manipulation 
import pandas as pd
import numpy as np

##for data visualizations and distributions of the variables
import matplotlib as mtb
import matplotlib.pyplot as plt
from pandas_summary import DataFrameSummary
import scipy as sc
import sklearn as skl

# Import dataset

In [2]:
#import pandas as pd

#import sendy data into 
#xls = pd.ExcelFile('../Sendy_data/Data_Analyst_Data_for_Case_Study.xlsx')

#query_results_df = pd.read_excel(xls, 'Query_result')
#missed_cancellation = pd.read_excel(xls, 'Missing_Cancel_Reason')

In [3]:
#read the data file
query_df = pd.read_csv('../Sendy_data/query_result.csv', sep=',')
missed_cancellation_df = pd.read_csv('../Sendy_data/missing_cancellation_reason.csv', sep=',')


In [4]:
query_df.head()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
0,Personal,NaN,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,17:55:38,NaN,NaN,1.0,17:56:35,9.0,-1.313053,36.841437,OnDemAND
1,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,12:26:27,1.0,12:27:36,1.0,12:28:44,14.0,-1.318959,36.853394,OnDemAND
2,Business,20680.0,Bike,Express,android,I placed the wrong locations.,Customer,1.0,13:21:18,1.0,13:21:34,1.0,13:47:29,16.0,-1.313854,36.706867,OnDemAND
3,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,10:29:19,NaN,NaN,1.0,10:29:29,20.0,-1.206197,36.778776,OnDemAND
4,Personal,NaN,Bike,Standard,ios,No Partner,Staff,1.0,14:34:05,1.0,14:59:08,1.0,15:49:41,11.0,-1.296958,36.808166,OnDemAND


In [5]:
missed_cancellation_df.head()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
0,Personal,NaN,Bike,Express,ios,NaN,Customer,1.0,15:23:44,1.0,16:40:47,1.0,16:43:50,4.0,-1.299370,36.822107,OnDemAND
1,Business,20680.0,Bike,Express,desktop,NaN,Staff,1.0,14:21:46,1.0,14:22:20,1.0,14:30:48,17.0,-1.264490,36.789303,OnDemAND
2,Business,17968.0,Bike,Express,desktop,NaN,Customer,1.0,15:13:20,NaN,NaN,1.0,15:56:34,18.0,-1.427056,36.961499,OnDemAND
3,Personal,NaN,Bike,Standard,android,NaN,Customer,1.0,13:54:19,NaN,NaN,1.0,13:56:41,8.0,-1.301737,36.743936,OnDemAND
4,Personal,NaN,Bike,Standard,android,NaN,Staff,1.0,13:24:05,1.0,15:57:20,1.0,17:14:46,13.0,-1.308098,36.920464,Scheduled


In [6]:
query_df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Personal or Business', 'Business Number', 'Vehicle Type',
       'Standard or Express', 'Platform', 'Cancel Reason', 'Cancelled by',
       'Placement Day', 'Placement Time', 'Confirmation Day',
       'Confirmation Time', 'Cancellation Day', 'Cancellation Time',
       'Distance (KM)', 'Pickup Lat', 'Pickup Long', 'scheduled'],
      dtype='object')>

## Rename columns

In [7]:
# Rename columns using a dictionary to map values

query_df.rename(columns={'Personal or Business':'personal_or_Business', 'Business Number':'Business_Number', 'Vehicle Type':'Vehicle_Type', 
                                 'Standard or Express':'Standard_or_Express', 'Cancel Reason':'Cancel_Reason', 'Cancelled by':'Cancelled_by',
                                 'Placement Day':'Placement_Day', 'Placement Time':'placement_time', 'Confirmation Day':'Confirmation_Day',
                                'Confirmation Time':'Confirmation_Time', 'Cancellation Day':'Cancellation_Day', 'Cancellation Time':'Cancellation_time',
                                'Distance (KM)':'Distance_(KM)', 'Pickup Lat':'Pickup_Lat', 'Pickup Long':'Pickup_Long'}, inplace=True)

In [8]:
query_df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['personal_or_Business', 'Business_Number', 'Vehicle_Type',
       'Standard_or_Express', 'Platform', 'Cancel_Reason', 'Cancelled_by',
       'Placement_Day', 'placement_time', 'Confirmation_Day',
       'Confirmation_Time', 'Cancellation_Day', 'Cancellation_time',
       'Distance_(KM)', 'Pickup_Lat', 'Pickup_Long', 'scheduled'],
      dtype='object')>

In [13]:
query_df.columns = map(str.lower, query_df.columns)

In [14]:
query_df.columns

Index(['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled'],
      dtype='object')

In [15]:
query_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6406 entries, 0 to 6405
Data columns (total 17 columns):
personal_or_business    6398 non-null object
business_number         3064 non-null float64
vehicle_type            6398 non-null object
standard_or_express     6393 non-null object
platform                6398 non-null object
cancel_reason           6398 non-null object
cancelled_by            6398 non-null object
placement_day           6398 non-null float64
placement_time          6398 non-null object
confirmation_day        2882 non-null float64
confirmation_time       2882 non-null object
cancellation_day        6398 non-null float64
cancellation_time       6398 non-null object
distance_(km)           6398 non-null float64
pickup_lat              6398 non-null float64
pickup_long             6398 non-null float64
scheduled               6398 non-null object
dtypes: float64(7), object(10)
memory usage: 850.9+ KB


The total number of entries is 6398 with both categorical and numerical data. 

## Change data types 

In [17]:
query_df['cancellation_day'].value_counts()

5.0    1308
4.0    1188
3.0    1056
2.0    1045
1.0     832
6.0     661
7.0     308
Name: cancellation_day, dtype: int64

## Check for Nan

In [ ]:
query_df.isnull().any()

In [ ]:
query_df.isnull().sum()